# FRETpredict tutorial (pp11)

In [31]:
import MDAnalysis
import numpy as np
import os
import pandas as pd
import pickle
import yaml
from FRETpredict import FRETpredict
import warnings

warnings.filterwarnings('ignore')

### Quick biological background

Polyproline 11 (pp11) has been described as behaving like a rigid rod, and was used as a "spectroscopic ruler" in the seminal paper by Stryer and Haugland. The pp11 system is a classical example of the importance of comparing molecular models with FRET data.

![title](pp11_structure.png)

We will perform FRET Efficiency calculations placing the rotamer libraries on the extremal residues.<br>
<br>First, let's have a look at the possible rotamer libraries we can use and how they're called.

### Rotamer library

In [2]:
with open('../../FRETpredict/lib/libraries.yml') as f:
    libraries = yaml.load(f, Loader=yaml.FullLoader)
#libraries

Every Rotamer Library name is composed of three parts: the manufacturer (AlexaFluor, ATTO, Lumiprobe), the peak wavelength (e.g. 488, 550, 647), and the linker that connects the dye to the protein (C1R, C2R, C3R, L1R, L2R, B1R).<br>
<br>To learn more about rotamer libraries, see __[`Tutorial_generate_new_rotamer_libraries`](https://github.com/Monte95/FRETpredict/blob/50ad48c7f2df4fc0aaca52158eb349cc82e4e2c1/tests/tutorials/Tutorial_generate_new_rotamer_libraries.ipynb)__.

### FRET efficiency calculation

Now, we will select the parameters for the FRET Efficiency calculation: 
- The `residues` to place the rotamer libraries on and their protein `chain`
For this tutorial we're going to use the first and the last residues of the pp11 chain. 

- The rotamer libraries we will use: AlexaFluor dyes 488 and 594, with C1R linkers.
`donor` and `acceptor` are used for R0 calculations, while `libname_1` and `libname_2` for the FRET Efficiency calculation. `r0lib` is the path to the dyes files (by default set to `lib/R0/`).
- The Universe object for the `protein` structure.
- `electrostatic` calculations will be enabled, and the `temperature` will be set at 298K.

In [3]:
# Experimental FRET efficiency value to compare our results
Ex = 0.88

# Create MDAnalysis.Universe object for the protein 
u = MDAnalysis.Universe('../test_systems/pp11/pp11.pdb', '../test_systems/pp11/pp11.xtc')

Let's create an instance of the FRETpredict class.

In [4]:
FRET = FRETpredict(protein=u, residues=[0, 12], temperature=298, 
                   chains=['A', 'A'], electrostatic=True,
                   donor='AlexaFluor 488', acceptor='AlexaFluor 594', 
                   libname_1='AlexaFluor 488 C1R cutoff10',
                   libname_2='AlexaFluor 594 C1R cutoff10',
                   output_prefix='test/E_pp11_10')

Run FRET efficiency calculations.

In [5]:
FRET.run()

1/316-2/316-3/316-4/316-5/316-6/316-7/316-8/316-9/316-10/316-11/316-12/316-13/316-14/316-15/316-16/316-17/316-18/316-19/316-20/316-21/316-22/316-23/316-24/316-25/316-26/316-27/316-28/316-29/316-30/316-31/316-32/316-33/316-34/316-35/316-36/316-37/316-38/316-39/316-40/316-41/316-42/316-43/316-44/316-45/316-46/316-47/316-48/316-49/316-50/316-51/316-52/316-53/316-54/316-55/316-56/316-57/316-58/316-59/316-60/316-61/316-62/316-63/316-64/316-65/316-66/316-67/316-68/316-69/316-70/316-71/316-72/316-73/316-74/316-75/316-76/316-77/316-78/316-79/316-80/316-81/316-82/316-83/316-84/316-85/316-86/316-87/316-88/316-89/316-90/316-91/316-92/316-93/316-94/316-95/316-96/316-97/316-98/316-99/316-100/316-101/316-102/316-103/316-104/316-105/316-106/316-107/316-108/316-109/316-110/316-111/316-112/316-113/316-114/316-115/316-116/316-117/316-118/316-119/316-120/316-121/316-122/316-123/316-124/316-125/316-126/316-127/316-128/316-129/316-130/316-131/316-132/316-133/316-134/316-135/316-136/316-137/316-138/316-139/

Save the FRETpredict object to `pickle` file for future use.

In [6]:
with open('test/FRET_pp11_obj.pkl', 'wb') as file:
    
    pickle.dump(FRET, file)
    
    print('Object successfully saved to "FRET_pp11_obj.pkl"')

Object successfully saved to "FRET_pp11_obj.pkl"


Create DataFrame of the data (experimental and predicted) and show the results.

In [24]:
results = []

data = pd.read_pickle('test/E_pp11_10-data-0-12.pkl')

E_avg = np.mean([float(data.loc['Estatic', 'Average']), 
                 float(data.loc['Edynamic1', 'Average']),
                 float(data.loc['Edynamic2', 'Average'])])

results.append({'res': '1-13',
                'chromophore': 'AlexaFluor 488 C1R - AlexaFluor 594 C1R ',
                'cutoff' : 10,
                'k2': float(data.loc['k2', 'Average']),
                'Ex': Ex, 
                'Es': float(data.loc['Estatic', 'Average']),
                'Ed': float(data.loc['Edynamic1', 'Average']),
                'Ed2': float(data.loc['Edynamic2', 'Average']), 
                'E_avg': E_avg})
    
# Save data
np.save('test/results_pp11.npy', results)

# Display results
results_pp11_df = pd.DataFrame(results,index=['Calc. R0'])
results_pp11_df

,res,chromophore,cutoff,k2,Ex,Es,Ed,Ed2,E_avg
Calc. R0,1-13,AlexaFluor 488 C1R - AlexaFluor 594 C1R,10,0.684587,0.88,0.731976,0.876376,0.993194,0.867182


## R0 calculation

It is possible to select the R0 calculation method for the FRET Efficiency calculation.<br>
<br>By default (as done with the previous example), R0 is calculated for the provided dyes pair (`donor` and `acceptor`) for every rotamer conformation, taking the relative orientation into account.<br>
<br>However, it is possible to select a fixed R0 value. The only change is in the parameters passed to FRETpredict.<br>
<br>Let's see how it's done.

In [8]:
FRET_fixedR0 = FRETpredict(protein=u, residues=[0, 12], temperature=298, 
                           chains=['A', 'A'], 
                           fixed_R0=True, r0=5.68, electrostatic=True,
                           libname_1='AlexaFluor 488 C1R cutoff10',
                           libname_2='AlexaFluor 594 C1R cutoff10', 
                           output_prefix='test/E_pp11_10_fixedR0')

We set `fixed_R0=True` and passed to the `r0` parameter the selected R0 value. __[`lib/R0/R0_pairs.csv`](https://github.com/Monte95/FRETpredict/blob/ce327c60cf5f86a33251b2e0c60cf1f668bc46e2/FRETpredict/lib/R0/R0_pairs.csv)__ reports the R0 values for many dye pairs.<br>
<br>Otherwise, if the R0 value is not present in the file, online services as __[FPBase](https://www.fpbase.org/fret/)__ can be used to obtain it.

Now let's run FRET Efficiency calculations.

In [9]:
FRET_fixedR0.run()

1/316-2/316-3/316-4/316-5/316-6/316-7/316-8/316-9/316-10/316-11/316-12/316-13/316-14/316-15/316-16/316-17/316-18/316-19/316-20/316-21/316-22/316-23/316-24/316-25/316-26/316-27/316-28/316-29/316-30/316-31/316-32/316-33/316-34/316-35/316-36/316-37/316-38/316-39/316-40/316-41/316-42/316-43/316-44/316-45/316-46/316-47/316-48/316-49/316-50/316-51/316-52/316-53/316-54/316-55/316-56/316-57/316-58/316-59/316-60/316-61/316-62/316-63/316-64/316-65/316-66/316-67/316-68/316-69/316-70/316-71/316-72/316-73/316-74/316-75/316-76/316-77/316-78/316-79/316-80/316-81/316-82/316-83/316-84/316-85/316-86/316-87/316-88/316-89/316-90/316-91/316-92/316-93/316-94/316-95/316-96/316-97/316-98/316-99/316-100/316-101/316-102/316-103/316-104/316-105/316-106/316-107/316-108/316-109/316-110/316-111/316-112/316-113/316-114/316-115/316-116/316-117/316-118/316-119/316-120/316-121/316-122/316-123/316-124/316-125/316-126/316-127/316-128/316-129/316-130/316-131/316-132/316-133/316-134/316-135/316-136/316-137/316-138/316-139/

Save the FRETpredict object to `pickle` file for future use.

In [10]:
with open('test/FRET_pp11_fixedR0_obj.pkl', 'wb') as file:
    
    pickle.dump(FRET, file)
    
    print('Object successfully saved to "FRET_pp11_fixedR0_obj.pkl"')

Object successfully saved to "FRET_pp11_fixedR0_obj.pkl"


Create DataFrame of the data (experimental and predicted) and show the results.

In [25]:
results_fixedR0 = []

data = pd.read_pickle('test/E_pp11_10_fixedR0-data-0-12.pkl')

E_avg = np.mean([float(data.loc['Estatic', 'Average']), 
                 float(data.loc['Edynamic1', 'Average']),
                 float(data.loc['Edynamic2', 'Average'])])

results_fixedR0.append({'res': '1-13',
                        'chromophore': 'Fixed R0 AlexaFluor 488 C1R - AlexaFluor 594 C1R ',
                        'cutoff' : 10,
                        'k2': float(data.loc['k2', 'Average']),
                        'Ex': Ex, 
                        'Es': float(data.loc['Estatic', 'Average']),
                        'Ed': float(data.loc['Edynamic1', 'Average']),
                        'Ed2': float(data.loc['Edynamic2', 'Average']), 
                        'E_avg': E_avg})
    
# Save data
np.save('test/results_fixedR0.npy', results_fixedR0)

# Display results and compare with previous case
results_fixedR0_df = pd.DataFrame(results_fixedR0, index=['Fixed R0'])
pd.concat([results_pp11_df, results_fixedR0_df])

,res,chromophore,cutoff,k2,Ex,Es,Ed,Ed2,E_avg
Calc. R0,1-13,AlexaFluor 488 C1R - AlexaFluor 594 C1R,10,0.684587,0.88,0.731976,0.876376,0.993194,0.867182
Fixed R0,1-13,Fixed R0 AlexaFluor 488 C1R - AlexaFluor 594 C1R,10,0.684587,0.88,0.729104,0.874128,0.992961,0.865398


## Reweighting

As you probably have noticed, at the end of its calculations `FRET.run()` prompted the `effective fraction of frames contributing to average: 0.9728538649554426`.

This is an indication for the usefulness of reweighting the frames of the trajectory. Each frame is in fact assigned a weight $w_s$ based on dye-protein interactions obtained by multiplying the Boltzmann partition function $Z_{si}$ of the donor and acceptor for that frame.<br>

$w_s = \frac{Z_s}{\sum_s (Z_s)}$, with $Z_s = Z_{s1} \cdot Z_{s2}$. Weights are normalized such that $\sum_s w_s = 1$.

In this way, frames with many dye-protein steric clashes have a low weight.

The effective fraction of frames contributing to average is thus computed as $\phi_{eff} = \exp(S)$ with $S = - \sum_s w_s \cdot \ln(w_s)$

FRETpredict calculations are run by default with all $w_s = 1 /$num_frames.

Based on the $\phi_{eff}$ value, one could decide to reweight the trajectory frames based on $w_s$. FRETpredict implement an easy-to-use reweighting approach.

In [12]:
# REWEIGHTING
FRET.reweight(reweight_prefix='test/E_pp11_10_reweighted')

Effective fraction of frames contributing to average: 0.9728538434331249


If you want to combine dye-protein interaction weights with weights obtained from different calculations (e.g, Enhanced sampling simulations) 

In [13]:
user_weights_pp11 = np.load('test/user_weights.npy', allow_pickle=True)

FRET.reweight(reweight_prefix='test/E_pp11_10_reweighted', user_weights=user_weights_pp11)

Effective fraction of frames contributing to average: 0.8012029686522127


Otherwise, if you want to reweight a previously used trajectory

In [14]:
# Load FRETpredict object from pickle file
with open('test/FRET_pp11_obj.pkl', 'rb') as file:
    
    FRET_file = pickle.load(file)
    
    print(f'Object successfully loaded from "FRET_pp11_obj.pkl"')

Object successfully loaded from "FRET_pp11_obj.pkl"


In [15]:
# REWEIGHTING
FRET_file.reweight(reweight_prefix='test/E_pp11_10_reweighted')

Effective fraction of frames contributing to average: 0.9728538434331249


If you want to use custom pre-computed weights and directly obtain the reweighted FRET efficiencies, you can pass the custom weights as a `numpy.array` to the FRETpredict class throught the `user_weights` parameter, and they will be automatically combined with the dye-protein interaction weights:

In [19]:
user_weights_pp11 = np.load('test/user_weights.npy')

FRET = FRETpredict(protein=u, residues=[0, 12], temperature=298, 
                   chains=['A', 'A'], electrostatic=True,
                   donor='AlexaFluor 488', acceptor='AlexaFluor 594', 
                   libname_1='AlexaFluor 488 C1R cutoff10',
                   libname_2='AlexaFluor 594 C1R cutoff10',
                   r0lib='../../FRETpredict/lib/R0/',
                   user_weights=user_weights_pp11,
                   output_prefix='test/E_pp11_10_reweighted')

Create the DataFrame of the data (experimental and predicted) and show the results.

In [28]:
results_ws = []

data = pd.read_pickle('test/E_pp11_10_reweighted-data-0-12.pkl')
    
E_avg = np.mean([float(data.loc['Estatic', 'Average']), 
                 float(data.loc['Edynamic1', 'Average']),
                 float(data.loc['Edynamic2', 'Average'])])

results_ws.append({'res': '1-13',
                   'chromophore': 'AlexaFluor 488 C1R - AlexaFluor 594 C1R ',
                   'cutoff' : 10,
                   'k2': float(data.loc['k2', 'Average']),
                   'Ex': Ex,
                   'Es': float(data.loc['Estatic', 'Average']),
                   'Ed': float(data.loc['Edynamic1', 'Average']),
                   'Ed2': float(data.loc['Edynamic2', 'Average']), 
                   'E_avg': E_avg,})

# Save data
np.save('test/results_pp11_ws.npy', results_ws)

# Display results
results_pp11_ws_df = pd.DataFrame(results_ws,index=['Reweighted'])
pd.concat([results_pp11_df, results_pp11_ws_df])

,res,chromophore,cutoff,k2,Ex,Es,Ed,Ed2,E_avg
Calc. R0,1-13,AlexaFluor 488 C1R - AlexaFluor 594 C1R,10,0.684587,0.88,0.731976,0.876376,0.993194,0.867182
Reweighted,1-13,AlexaFluor 488 C1R - AlexaFluor 594 C1R,10,0.690553,0.88,0.736478,0.880143,0.993216,0.869946


In this case, with pp11 a small change is expected from the global weighting because the it is essentially rigid, and also the structure should already be "adapted" to the dyes. This is reflected in the hight percentage of frames contributing to the average (~97%).

Reweighting is more useful when this percentage is lower, as it is usually the case for Intrinsically Disordered Proteins (see the FRETpredict paper https://doi.org/10.1101/2023.01.27.525885).